In [39]:
import numpy as np;
import os;
import util;
import visualize;
import random;

def comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre=None):
    img_files_all=[];
    captions_all=[];
    if img_caption_pre is not None:
        assert len(img_caption_pre)==len(img_dirs);
    batch_str=True;
    if range_batches is None:
        range_batches=range(1);
        batch_str=False;
    
    for batch_num in range_batches:
    # range(1,num_batches+1):
        for im_num in range_images:
            for idx_img_dir,img_dir in enumerate(img_dirs):
                files_curr_list=[];
                captions_curr_list=[];
            
                if loss_post is None:
                    loss_str='';
                else:
                    loss_all=np.load(os.path.join(img_dir,str(batch_num)+loss_post));
                    if im_num>loss_all.shape[0]:
                        continue;
                    loss_curr=loss_all[im_num-1,0];
                    loss_str="{:10.4f}".format(loss_curr);
                if not batch_str:
                    files_curr=[os.path.join(img_dir,str(im_num)+file_post_curr) for file_post_curr in file_post];
                else:
                    files_curr=[os.path.join(img_dir,str(batch_num)+'_'+str(im_num)+file_post_curr) for file_post_curr in file_post];
                files_curr=[util.getRelPath(file_curr,dir_server) for file_curr in files_curr];
                captions_curr=[os.path.split(file_curr)[1]+' '+loss_str for file_curr in files_curr];
                if img_caption_pre is not None:
                    captions_curr=[img_caption_pre[idx_img_dir]+' '+caption_curr for caption_curr in captions_curr];
                files_curr_list.extend(files_curr);
                captions_curr_list.extend(captions_curr);
                
                img_files_all.append(files_curr_list);
                captions_all.append(captions_curr_list);
            
    visualize.writeHTML(out_file_html,img_files_all,captions_all,224,224);
    

def parseLogFile(log_file,gt_loss=False):
    log_data=util.readLinesFromFile(log_file);
    final_loss_str='minibatches processed: all'
    final_loss=[line_curr for line_curr in log_data if final_loss_str in line_curr];
    assert len(final_loss)==1;
    final_loss=final_loss[0].split(' ');
    final_loss=final_loss[-1].strip('"');
    gt_str='gt val loss'

    if gt_loss:
        gt_losses=[line_curr for line_curr in log_data if gt_str in line_curr];
        gt_losses=[line_curr.split(' ')[-1].strip('"') for line_curr in gt_losses];
        gt_losses=' '.join(gt_losses);
    
    if gt_loss:
        return final_loss,gt_losses
    else:
        return final_loss;


def getCommandTestTrainImages(path_to_th,
                              outDir,
                              num_ctrl_pts,
                              val_horse_data_path,
                              val_human_data_path,
                             out_dir_images):
    command=['th',path_to_th];
    command.extend(['-outDir',outDir]);
    command.extend(['-model',os.path.join(outDir,'final','model_all_final.dat')])
    command.extend(['-num_ctrl_pts',str(num_ctrl_pts)]);
    command.extend(['-out_dir_images',out_dir_images]);
    command.extend(['-val_horse_data_path',val_horse_data_path]);
    command.extend(['-val_human_data_path',val_human_data_path]);
    command=' '.join(command);
    return command;
 



    

In [31]:

dirs_meta=['/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr',
           '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr/epoch_4',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_36_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_declr/quick_decrease',
              '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64',
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'];
num_ctrl_pts=[25,25,36,64,64,64,36];
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_tps_var_lr.th'
out_dir_images='train_images';
path_common='/home/SSD3/maheen-data/horse_project/data_check'

# val_horse_data_path=os.path.join(path_common,'horse/matches_5_val_allKP_minLoss_clean.txt');
# val_human_data_path=os.path.join(path_common,'aflw/matches_5_val_allKP_minLoss_noIm_clean.txt');

val_horse_data_path=os.path.join(path_common,'horse','matches_5_train_allKP_minLoss_clean_subset.txt')
val_human_data_path=os.path.join(path_common,'aflw','matches_5_train_allKP_minLoss_noIm_clean_subset.txt')

commands=[];
out_file_commands=os.path.join(dirs_meta[0],'test_on_train.sh');
for num_ctrl_pts,outDir in zip(num_ctrl_pts,dirs_meta):
    commands.append(getCommandTestTrainImages(path_to_th,outDir,num_ctrl_pts,\
                                              val_horse_data_path,val_human_data_path,out_dir_images));
print out_file_commands
util.writeFile(out_file_commands,commands);


    


/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr/test_on_train.sh


In [21]:


horse_data_path=os.path.join(path_common,'horse','matches_5_train_allKP_minLoss_clean.txt')
human_data_path=os.path.join(path_common,'aflw','matches_5_train_allKP_minLoss_clean.txt')
human_data_path_noIm=os.path.join(path_common,'aflw','matches_5_train_allKP_minLoss_noIm_clean.txt')

files=[horse_data_path,human_data_path,human_data_path_noIm];
out_files=[file_curr[:file_curr.rindex('.')]+'_subset.txt' for file_curr in files];

data_horse=util.readLinesFromFile(horse_data_path);
idx=range(len(data_horse));
random.shuffle(idx);
to_select=193;
idx=idx[:to_select];


for in_file,out_file in zip(files,out_files):
    data_curr=util.readLinesFromFile(in_file);
    data_out=[data_curr[idx_curr] for idx_curr in idx]
    print len(data_curr),len(data_out);
    print out_file
    util.writeFile(out_file,data_out);




3531 193
/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_clean_subset.txt
3531 193
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_clean_subset.txt
3531 193
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_noIm_clean_subset.txt


In [33]:
dirs_meta=['/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr',
              '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr/epoch_4',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_36_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_declr/quick_decrease',
              '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64',
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'];

dirs_strs=['25 small dec','25 small dec diff','36 small dec','64 small dec','64 big dec','64 big nodec','orig 36 small nodec']

test_dirs=[os.path.join(dir_curr,'test_images') for dir_curr in dirs_meta];
log_files=[os.path.join(dir_curr,'log_test.txt') for dir_curr in test_dirs];
loss_ims=[os.path.join(dir_curr,'intermediate','loss_all.png') for dir_curr in dirs_meta];

losses=[parseLogFile(file_curr,True) for file_curr in log_files];
# for loss_curr,dir_curr in zip(losses,dirs_meta):
#     print loss_curr,dirs_meta;
    

out_file_html=os.path.join(dirs_meta[0],'comparativeLosses.html');
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'
imgs=[];
captions=[];
for idx,file_curr in enumerate(loss_ims):
    caption_curr=dirs_strs[idx]+' '+losses[idx][0];
    captions.append([caption_curr]);
    imgs.append([util.getRelPath(file_curr,dir_server)]);

visualize.writeHTML(out_file_html,imgs,captions);
print out_file_html.replace(dir_server,click_str)


http://vision1.idav.ucdavis.edu:1000/horse_project/tps_bn_clean_var_lr_25_small_declr/comparativeLosses.html


In [41]:
dirs_meta=['/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr',
           '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_36_small_declr',
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'];
img_dirs=[os.path.join(dir_curr,'test_images') for dir_curr in dirs_meta];
file_post=['_horse.jpg','_gtwarp.jpg','_predwarp.jpg',];
img_caption_pre=['25 small dec','36 small dec','orig 36 small nodec']
range_batches=[1,2];
range_images=range(1,101);
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000';
out_file_html=os.path.join(dirs_meta[0],'comparison_viz.html');
loss_post=None;
comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,
                   out_file_html,dir_server,img_caption_pre)
print out_file_html.replace(dir_server,click_str);



http://vision1.idav.ucdavis.edu:1000/horse_project/tps_bn_clean_var_lr_25_small_declr/comparison_viz.html
